# Bot Trades on Hyperdrive Contracts

In [ ]:
# test: skip-notebook
from __future__ import annotations
import logging

from numpy.random._generator import Generator as NumpyGenerator

import elfpy.agents.agent as agent
import elfpy.agents.policies.random_agent as random_agent
import elfpy.markets.hyperdrive.hyperdrive_market as hyperdrive_market
import elfpy.simulators as simulators
import elfpy.utils.sim_utils as sim_utils
import elfpy.utils.outputs as output_utils
import elfpy.utils.apeworx_integrations as ape_utils
import elfpy.utils.post_processing as post_utils

import ape
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

 ### Setup experiment parameters

In [ ]:
config = simulators.Config()

config.title = "random bot demo"
config.pricing_model_name = "Hyperdrive"  # can be yieldspace or hyperdrive

config.num_trading_days = 5  # Number of simulated trading days
config.num_blocks_per_day = 10  # Blocks in a given day (7200 means ~12 sec per block)
config.num_position_days = 8
config.trade_fee_percent = 0.10  # fee percent collected on trades
config.redemption_fee_percent = 0.005  # 5 bps

num_agents = 4  # int specifying how many agents you want to simulate
agent_budget = 1_000_000  # max money an agent can spend
trade_chance = 5 / (
    config.num_trading_days * config.num_blocks_per_day
)  # on a given block, an agent will trade with probability `trade_chance`

config.target_fixed_apr = 0.01  # target fixed APR of the initial market after the LP
config.target_liquidity = 500_000_000  # target total liquidity of the initial market, before any trades

# Define the variable apr
config.variable_apr = [0.03] * config.num_trading_days

config.do_dataframe_states = True

config.log_level = output_utils.text_to_log_level("INFO")  # Logging level, should be in ["DEBUG", "INFO", "WARNING"]
config.log_filename = "random_bots"  # Output filename for logging

config.freeze()  # type: ignore

### Setup agents

In [ ]:
def get_example_agents(
    rng: NumpyGenerator, budget: int, new_agents: int, existing_agents: int = 0
) -> list[agent.Agent]:
    """Instantiate a set of custom agents"""
    agents = []
    for address in range(existing_agents, existing_agents + new_agents):
        agent = random_agent.Policy(
            rng=rng,
            trade_chance=trade_chance,
            wallet_address=address,
            budget=budget,
        )
        agent.log_status_report()
        agents += [agent]
    return agents


### Setup simulation objects

In [ ]:
# define root logging parameters
output_utils.setup_logging(log_filename=config.log_filename, log_level=config.log_level)

# get an instantiated simulator object
simulator = sim_utils.get_simulator(config)

### Run the simulation

In [ ]:
# add the random agents
rnd_agents = get_example_agents(
    rng=simulator.rng,
    budget=agent_budget,
    new_agents=num_agents,
    existing_agents=1,
)
simulator.add_agents(rnd_agents)
logging.info(
    "Simulator has %d agents with budgets =%s.",
    len(simulator.agents),
    [sim_agent.budget for sim_agent in simulator.agents.values()],
)

In [ ]:
# run the simulation
simulator.run_simulation()

# get the trade tape
sim_trades = simulator.new_simulation_state.trade_updates.trade_action.tolist()
logging.info(
    "User trades:\n%s", 
    "\n\n".join([f"{trade}" for trade in sim_trades]),
)

In [ ]:
sim_trades_df = post_utils.compute_derived_variables(simulator)

In [ ]:
for trade_idx in range(1, len(sim_trades_df)):
    fig, axs, _ = output_utils.get_gridspec_subplots()
    ax = axs[0]
    ax.step(sim_trades_df["trade_number"].iloc[1:trade_idx], sim_trades_df["shorts_outstanding"].iloc[1:trade_idx], label="Shorts outstanding")
    ax.step(sim_trades_df["trade_number"].iloc[1:trade_idx], sim_trades_df["longs_outstanding"].iloc[1:trade_idx], label="Longs outstanding")
    ax.set_xlabel("Trade number")
    ax.set_ylabel("Outstanding open positions")
    ax.set_title("Random longs & shorts")
    ax.set_xlim([1, sim_trades_df["trade_number"].iloc[-1]-2])
    y_max = round(max(max(sim_trades_df["shorts_outstanding"]), max(sim_trades_df["longs_outstanding"])))
    ax.set_ylim([0, y_max])
    ax.ticklabel_format(axis="both", style="sci")
    ax.legend();
    fig.savefig(f"./figs/sim_trade_balances/sim_random_trades_{trade_idx:02g}.png", bbox_inches="tight")
    plt.close(fig)

In [ ]:
fig, axs, _ = output_utils.get_gridspec_subplots()
ax = axs[0]
ax.step(sim_trades_df["trade_number"].iloc[1:-1], sim_trades_df["fixed_apr"].iloc[1:-1], label="APR")
ax.set_xlabel("Trade number")
ax.set_ylabel("Fixed APR")
ax.set_title("Random longs & shorts")
fig.savefig(f"./figs/fixed_apr.png", bbox_inches="tight")

In [ ]:
lp_trades = sim_trades_df.groupby("trade_number").agg(
    {
        f"agent_{0}_pnl": ["sum"]
    }
)
lp_trades.columns = ["_".join(col).strip() for col in lp_trades.columns.values]
lp_trades = lp_trades.reset_index()

In [ ]:
fig, axs, _ = output_utils.get_gridspec_subplots()
ax = axs[0]
ax.step(lp_trades["trade_number"].iloc[1:-1], lp_trades["agent_0_pnl_sum"].iloc[1:-1], label="PNL")
ax.set_xlabel("Trade number")
ax.set_ylabel("Agent 0 PNL share proceeds")
ax.set_title("Random longs & shorts")
fig.savefig(f"./figs/LP_PNL.png", bbox_inches="tight")

### Apeworx Network setup

In [ ]:
provider = ape.networks.parse_network_choice("ethereum:local:foundry").__enter__()
project_root = Path.cwd().parent.parent
project = ape.Project(path=project_root)

### Generate agent accounts

In [ ]:
governance = ape.accounts.test_accounts.generate_test_account()
sol_agents = {"governance": governance}
for agent_address, sim_agent in simulator.agents.items():
    sol_agent = ape.accounts.test_accounts.generate_test_account()  # make a fake agent with its own wallet
    sol_agent.balance = int(sim_agent.budget * 10**18)
    sol_agents[f"agent_{agent_address}"] = sol_agent

### Deploy contracts

In [ ]:
# use agent 0 to initialize the market
base_address = sol_agents["agent_0"].deploy(project.ERC20Mintable)
base_ERC20 = project.ERC20Mintable.at(base_address)

fixed_math_address = sol_agents["agent_0"].deploy(project.MockFixedPointMath)
fixed_math = project.MockFixedPointMath.at(fixed_math_address)

base_ERC20.mint(int(config.target_liquidity * 10**18), sender=sol_agents["agent_0"])

initial_supply = int(config.target_liquidity * 10**18)
initial_apr = int(config.target_fixed_apr * 10**18)
initial_share_price = int(config.init_share_price * 10**18)
checkpoint_duration = 86400  # seconds = 1 day
checkpoints_per_term = 365
position_duration_seconds = checkpoint_duration * checkpoints_per_term
time_stretch = int(1 / simulator.market.time_stretch_constant * 10**18)
curve_fee = int(config.trade_fee_percent * 10**18)
flat_fee = int(config.redemption_fee_percent * 10**18)
gov_fee = 0

hyperdrive_address = sol_agents["agent_0"].deploy(
    project.MockHyperdriveTestnet,
    base_ERC20,
    initial_apr,
    initial_share_price,
    checkpoints_per_term,
    checkpoint_duration,
    time_stretch,
    (curve_fee, flat_fee, gov_fee),
    governance,
)
hyperdrive = project.MockHyperdriveTestnet.at(hyperdrive_address)

with ape.accounts.use_sender(sol_agents["agent_0"]):
    base_ERC20.approve(hyperdrive, initial_supply)
    hyperdrive.initialize(initial_supply, initial_apr, sol_agents["agent_0"], False)


### Execute trades

In [ ]:
# get current block
genesis_block_number = ape.chain.blocks[-1].number
genesis_timestamp = ape.chain.provider.get_block(genesis_block_number).timestamp

# set the current block?
pool_state = [hyperdrive.getPoolInfo().__dict__]
pool_state[0]["block_number_"] = genesis_block_number
logging.info("pool_state=%s\n", pool_state)

sim_to_block_time = {}
trade_receipts = []
for trade in sim_trades:
    agent_key = f"agent_{trade.wallet.address}"
    trade_amount = int(trade.trade_amount * 10**18)
    logging.info(
        "agent_key=%s, action=%s, mint_time=%s",
        agent_key,
        trade.action_type.name,
        trade.mint_time,
    )
    if trade.action_type.name in ["ADD_LIQUIDITY", "REMOVE_LIQUIDITY"]:
        continue  # todo
    if trade.action_type.name == "OPEN_SHORT":
        with ape.accounts.use_sender(sol_agents[agent_key]):  # sender for contract calls 
            # Mint DAI & approve ERC20 usage by contract
            base_ERC20.mint(trade_amount)
            base_ERC20.approve(hyperdrive.address, trade_amount)
        new_state, trade_details = ape_utils.ape_open_position(
            hyperdrive_market.AssetIdPrefix.SHORT,
            hyperdrive,
            sol_agents[agent_key],
            trade_amount,
        )
        sim_to_block_time[trade.mint_time] = new_state["maturity_timestamp_"]
    elif trade.action_type.name == "CLOSE_SHORT":
        maturity_time = int(sim_to_block_time[trade.mint_time])
        new_state, trade_details = ape_utils.ape_close_position(
            hyperdrive_market.AssetIdPrefix.SHORT,
            hyperdrive,
            sol_agents[agent_key],
            trade_amount,
            maturity_time,
        )
    elif trade.action_type.name == "OPEN_LONG":
        with ape.accounts.use_sender(sol_agents[agent_key]):  # sender for contract calls 
            # Mint DAI & approve ERC20 usage by contract
            base_ERC20.mint(trade_amount)
            base_ERC20.approve(hyperdrive.address, trade_amount)
        new_state, trade_details = ape_utils.ape_open_position(
            hyperdrive_market.AssetIdPrefix.LONG,
            hyperdrive,
            sol_agents[agent_key],
            trade_amount,
        )
        sim_to_block_time[trade.mint_time] = new_state["maturity_timestamp_"]
    elif trade.action_type.name == "CLOSE_LONG":
        maturity_time = int(sim_to_block_time[trade.mint_time])
        new_state, trade_details = ape_utils.ape_close_position(
            hyperdrive_market.AssetIdPrefix.LONG,
            hyperdrive,
            sol_agents[agent_key],
            trade_amount,
            maturity_time,
        )
    else:
        raise ValueError(f"{trade.action_type=} must be opening or closing a long or short")
    trade_receipts.append(trade_details)
    new_state["action_type"] = trade.action_type.name
    new_state["trade_amount"] = trade_amount / 1e18
    new_state["agent_key"] = agent_key
    pool_state.append(new_state)

In [ ]:
trades_df = pd.DataFrame(pool_state)

In [ ]:
trades_df.columns

In [ ]:
for trade_idx in range(1, len(trades_df)):
    fig, axs, _ = output_utils.get_gridspec_subplots()
    ax = axs[0]
    ax.step(range(1, trade_idx), trades_df["shortsOutstanding_"].iloc[1:trade_idx] / 1e18, label="Shorts outstanding")
    ax.step(range(1, trade_idx), trades_df["longsOutstanding_"].iloc[1:trade_idx] / 1e18, label="Longs outstanding")
    ax.set_xlabel("Trade number")
    ax.set_ylabel("Outstanding open positions")
    ax.set_title("Random longs & shorts")
    ax.set_xlim([1, len(trades_df)])
    y_max = round(max(max(trades_df["shortsOutstanding_"]) / 1e18, max(trades_df["longsOutstanding_"]) / 1e18))
    ax.set_ylim([0, y_max])
    ax.ticklabel_format(axis="both", style="sci")
    ax.legend();
    fig.savefig(f"./figs/trade_balances/random_trades_{trade_idx:02g}.png", bbox_inches="tight")
    plt.close(fig)

In [ ]:
trades_df[["agent_key", "action_type", "trade_amount"]].iloc[1:]